In [11]:
import socket
from datetime import datetime

In [2]:
s = socket.socket(family=socket.AF_INET, type=socket.SOCK_RAW, proto=socket.IPPROTO_ICMP)

In [3]:
s.settimeout(3)

In [4]:
s.setsockopt(socket.IPPROTO_IP, socket.IP_TTL, 9)

In [5]:
header = b"\x08\x00\xf9\x21\xfe\xc6\x00\x17\x00\x00\x00\x00\x00\x00\x00\x00" \
b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
b"\x00\x00\x00\x00"

list(header[:8])

[8, 0, 249, 33, 254, 198, 0, 23]

In [6]:
s.sendto(header, ('8.8.8.8', 0))

52

In [49]:
ttl = 1
dest = '151.101.193.164'
while ttl < 12:
    s.setsockopt(socket.IPPROTO_IP, socket.IP_TTL, ttl)
    s.sendto(header, (dest, 0))
    start = datetime.now()
    try:
        resp = s.recv(1024)
    except socket.timeout as e:
        print(ttl, 'timed out')
    else:
        ms = (datetime.now() - start).total_seconds()*1000
        source = '.'.join([str(b) for b in resp[12:16]])
        print(ttl, source, round(ms, 7), 'ms')
        if source == dest:
            print('hello')
            break
    finally:
        ttl += 1

1 66.208.228.90 0.345 ms
2 192.168.86.1 0.026 ms
3 192.168.86.1 0.039 ms
4 10.1.10.1 0.085 ms
5 162.151.31.221 0.035 ms
6 96.120.90.121 0.022 ms
7 69.139.198.5 0.036 ms
8 68.85.57.241 0.023 ms
9 151.101.193.164 0.023 ms
hello


In [135]:
list(resp)[12:16]

[68, 86, 143, 93]

In [15]:
(datetime.now() - start).total_seconds()

90.804356

In [23]:
resp[12:16]

b'\xc0\xa8V\x01'